In [1]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob

### Import Output datas

In [7]:
def load_mgxs(sp_file_path,sm_file_path,model_file_path):
    sm=openmc.Summary(sm_file_path)
    sp = openmc.StatePoint(sp_file_path)
    sp.link_with_summary(sm)
    model=openmc.model.Model
    model=openmc.model.Model.from_model_xml(model_file_path)
    mgxs_lib = openmc.mgxs.Library(model.geometry)
    groups = openmc.mgxs.EnergyGroups(openmc.mgxs.GROUP_STRUCTURES['XMAS-172'])
    mgxs_lib.energy_groups = groups
    mgxs_lib.scatter_format = "legendre"
    mgxs_lib.legendre_order = 3
    mgxs_lib.mgxs_types = ['total', 'absorption', 'fission', 'nu-fission', 'chi', 'scatter matrix', 'nu-scatter matrix', 'inverse-velocity']
    mgxs_lib.tally_trigger =openmc.Trigger('std_dev', 1e-2)
    mgxs_lib.by_nuclide = False
    mgxs_lib.domain_type = 'cell'
    mgxs_lib.domains = model.geometry.get_all_material_cells().values()
    mgxs_lib.build_library()
    mgxs_lib.load_from_statepoint(sp)
    return(mgxs_lib)
    

In [23]:
sp_6_families_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/sp_6_families.h5'
sm_6_families_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/sm_6_families.h5'
model_6_families_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/model_6_families.xml'
mgxs_lib_6_families=load_mgxs(sp_6_families_path,sm_6_families_path,model_6_families_path)

sp_1_family_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/sp_1_family.h5'
sm_1_family_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/sm_1_family.h5'
model_1_family_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/model_1_family.xml'
mgxs_lib_1_family=load_mgxs(sp_1_family_path,sm_1_family_path,model_1_family_path)

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=7.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=8.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=9.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=109.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=110.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=111.
  warn(msg, IDWarning)
/

### Post Processing

In [24]:
def cell_domain(cell,mgxs_lib):
    for k in range(len(mgxs_lib.domains)):
        if mgxs_lib.domains[k].name==cell:
            return(mgxs_lib.domains[k].id)

In [25]:
def mgxs_value(cell,reaction,mgxs_lib):  
    cell_name=cell
    cell=cell_domain(cell_name,mgxs_lib)
    mgxs_plot = mgxs_lib.get_mgxs(cell,reaction)
    mgxs=mgxs_plot.get_pandas_dataframe(xs_type='macro')['mean'].values
    return(mgxs)

In [29]:
mgxs_value('fuel_1_family','fission',mgxs_lib_1_family)
mgxs_value('fuel_E','fission',mgxs_lib_6_families)

array([0.00000000e+00, 2.88028683e-02, 2.64185770e-02, 2.29087043e-02,
       2.28180820e-02, 2.29869855e-02, 2.18037421e-02, 1.70645210e-02,
       1.30820979e-02, 1.26746900e-02, 1.27560645e-02, 1.23813635e-02,
       1.24575217e-02, 1.26715901e-02, 1.25843564e-02, 1.14345797e-02,
       7.33690826e-03, 2.40447237e-03, 1.51493502e-03, 1.31093410e-03,
       1.19621960e-03, 9.61595012e-04, 8.02256394e-04, 7.67028597e-04,
       7.67325173e-04, 7.75786763e-04, 8.06023447e-04, 8.23132090e-04,
       8.30779536e-04, 8.52285416e-04, 8.88523665e-04, 9.57634378e-04,
       1.01071084e-03, 1.06015184e-03, 1.12309616e-03, 1.18635505e-03,
       1.23780055e-03, 1.30250145e-03, 1.36082372e-03, 1.40105701e-03,
       1.43154080e-03, 1.53769840e-03, 1.66196652e-03, 1.81662408e-03,
       2.03580601e-03, 2.07659147e-03, 2.28620801e-03, 2.59976032e-03,
       2.95708316e-03, 3.32143930e-03, 3.50275776e-03, 3.58459101e-03,
       4.26311083e-03, 3.83338087e-03, 5.21270941e-03, 5.91209447e-03,
      